In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [40]:
def find_delivery_time(list_of_info):
    time = 'N/A'
    for info in list_of_info:
        if "Min" in info:
            time = info
    return time

def find_delivery_fee(list_of_info):
    fee = 'N/A'
    for info in list_of_info:
        if "Delivery Fee" in info:
            fee = info
    return fee

def rewards(x):
    return True if x == 'Buy 1, Get 1 Free' else False

class RestaurantInfo:
    
    def __init__(self, info, link):
        self.deal = info[0]
        self.name = info[1]
        self.link = link
        self.fee = find_delivery_fee(info)
        self.time = find_delivery_time(info)
    
    @staticmethod
    def is_satisfied(info):
        return rewards(info[0])
    
    def print_info(self):
        print('The restaurant {0} has deal {1} with fee {2} and delivery time {3}'.format(self.name, self.deal, self.fee, self.time))

In [38]:
def get_all_restaurants(driver, my_location, category = 'Deals'):
    '''
    None = get_to_page(driver, my_location, category = 'Deals')
    
    Perform a series of simulated clicks on website to get to desired Uber Eats page.
    
    @param driver: WebDriver
        WebDriver object from Selenium.
    @param my_location: str
        User location input on ubereats.com.
    @param category: str
        The category user want to check.
    '''
    # Wait 10 seconds maximum to locate the web elements.
    location_pos = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="location-typeahead-home-input"]'))
    )
    find_food_pos = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="main-content"]/div[1]/div[2]/div/div[1]/button'))
    )
    
    # Perform series of actions to type my location in the text box
    # and redirect to the new page.
    actions = ActionChains(driver)
    actions.send_keys_to_element(location_pos, my_location)
    actions.pause(1)
    actions.click(find_food_pos)
    actions.perform()
    
    # If the category can be found in the pre_defined dictionary,
    # use the value in the dictionary as xpath.
    if category == 'Deals':
        nav_pos = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'nav'))
        )
        deals_pos = nav_pos.find_element_by_tag_name("li")
        deals_pos.click()
    else:
        search_pos = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'search-suggestions-typeahead-input'))
        )
        actions = ActionChains(driver)
        actions.send_keys_to_element(search_pos, category)
        actions.key_down(Keys.ENTER).key_up(Keys.ENTER)
        actions.perform()
    
    # Wait until all elements are present in the list of restaurants.
    list_of_restaurants = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id="main-content"]/div/div/div[2]/div/div[2]'))
    )
    list_of_restaurants = list_of_restaurants[0]
    
    # Use the fact that all restaurants have only one figure to find number of restaurants in the page.
    figures = WebDriverWait(driver, 10).until(
        EC.visibility_of_all_elements_located((By.TAG_NAME, 'figure'))
    )
    
    num_of_restaurants = len(figures)
    restaurants = []
    
    for i in range(num_of_restaurants):
        WebDriverWait(driver, 10).until(
            EC.text_to_be_present_in_element((By.XPATH, '//*[@id="main-content"]/div/div/div/div/div[2]/div[{0}]'.format(i + 1)),
                                          ' ')
        )
        item = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="main-content"]/div/div/div/div/div[2]/div[{0}]'.format(i + 1)))
        )
        item_info = item.text.split('\n')
        link_info = item.find_element_by_tag_name('a')
        link = link_info.get_attribute('href')
        
        if RestaurantInfo.is_satisfied(item_info):
            restaurants.append(RestaurantInfo(item_info, link))
    return restaurants

In [39]:
driver = webdriver.Chrome('C:\WebDriver\chromedriver.exe')
driver.maximize_window()
driver.get('https://www.ubereats.com')

get = get_all_restaurants(driver, '181 woodsworth road', 'Bubble Tea')
for res in get:
    res.print_info()

The restaurant Shuyi Tealicious 书亦烧仙草 (North York) has deal Buy 1, Get 1 Free with fee $2.49 and delivery time 45–55
The restaurant Tea Shop 168 (180 Steeles Ave W) has deal Buy 1, Get 1 Free with fee $4.49 and delivery time 30–40
The restaurant Tsaa Tea Shop has deal Buy 1, Get 1 Free with fee $6.49 and delivery time 35–45
The restaurant The Dessert Kitchen (Scarborough) has deal Buy 1, Get 1 Free with fee $5.49 and delivery time 40–50
The restaurant Sugar Marmalade (North York) has deal Buy 1, Get 1 Free with fee $2.49 and delivery time 30–40
The restaurant Sugar Marmalade (Midland) has deal Buy 1, Get 1 Free with fee $4.49 and delivery time 30–40
The restaurant Go for Tea (Downtown Markham) has deal Buy 1, Get 1 Free with fee $5.49 and delivery time 35–45
The restaurant Gao’s Braised Chicken Rice 高时记黄焖鸡 (37 Sheppard Ave E) has deal Buy 1, Get 1 Free with fee $2.49 and delivery time 35–45


In [41]:
for res in get:
    print(res.link)

https://www.ubereats.com/ca/toronto/food-delivery/shuyi-tealicious-%E4%B9%A6%E4%BA%A6%E7%83%A7%E4%BB%99%E8%8D%89-north-york/Ww473q4YTyGr4XJ1oCQTaA
https://www.ubereats.com/ca/toronto/food-delivery/tea-shop-168-180-steeles-ave-w/eIJ32ShTTa279Tzv2p4Eqg
https://www.ubereats.com/ca/toronto/food-delivery/tsaa-tea-shop/5-GZ80GRQoyRxSWzf8sLJw
https://www.ubereats.com/ca/toronto/food-delivery/the-dessert-kitchen-scarborough/O26Yi6yFTvK1_Q_BTsDEXQ
https://www.ubereats.com/ca/toronto/food-delivery/sugar-marmalade-north-york/YjoAAekoT9WZ-9LlNvt-4A
https://www.ubereats.com/ca/toronto/food-delivery/sugar-marmalade-midland/wzGRh4mNT3uXxSJOC-0JPQ
https://www.ubereats.com/ca/toronto/food-delivery/go-for-tea-downtown-markham/EBVSnCMDROGMPnkEN50_cA
https://www.ubereats.com/ca/toronto/food-delivery/gaos-braised-chicken-rice-%E9%AB%98%E6%97%B6%E8%AE%B0%E9%BB%84%E7%84%96%E9%B8%A1-37-sheppard-ave-e/W1u9h5lAQ8StyzV12QTHyQ


In [43]:
test = [1, 2, 3, 4, 5]

for i in range(10):
    print('i = {0}'.format(i))
    try:
        print(test[i])
    except:
        continue

i = 0
1
i = 1
2
i = 2
3
i = 3
4
i = 4
5
i = 5
i = 6
i = 7
i = 8
i = 9
